In [2]:
# This Jupyter Notbook is created by ZHANG Jinge. Jinge.Zhang@obspm.fr
# 05/12/2024, Observatoire de Paris site de Meudon.

# This file is for automatically load and read the existing .spectra data of Cas_A, under '/nenufar-ft/LT11' folder in nancep server.
# Then read and load the corresponding Cas_A's NenuFAR pointing data from '/nenufar/LT11'.

In [5]:

# Import all the packages we need in this Jupyter notebook.

from nenupy.io.tf import Spectra, TFTask   # run in python3.8.10
import astropy.units as u
from astropy.coordinates import SkyCoord

from nenupy.io.bst import BST
from nenupy.astro.pointing import Pointing

import numpy as np
import matplotlib.pyplot as plt

import os
import scandir
import glob

import scipy
# import AntPat
# import dreamBeam

import subprocess

In [ ]:
# This is creating lists of directory paths for .spectra data files and BST.fits data files for Cas_A through all LT11 observations.
# However, there are missing datas from .spectra files
# The missing date are:
# miss_days= ["20230226" "20230719" "20230722" "20230723" "20230724" "20230725" "20230726" "20230727"]

# The python Line Comprehension is wired when taking out more than 1 string from a list,
# so I'm not using the data from 2023/ 03 or 07. According the the missing day list above.

# I also found some of the days missing *_1.spectra. Which means no high frequency data avaliable.
# To be checked, does any *0_spectra files missed? According to the files number, it shoud be fine.

glob_spectra_path_list_raw=glob.glob("/databf/nenufar-tf/LT11/*/*/*_CAS_A_TRACKING/*1.spectra")
print(len(glob_spectra_path_list_raw),glob_spectra_path_list_raw)
# Here after LT11, it comes to 'year', 'month'. So can be chose the spcific month you want to analyse.
# the 0 (or 1) before the .spectra can be modified as well. 0 for low frequency, 1 for upper frequency.
# Use sp.pipeline.parameters.info() to check the information. use one of the .spectra file for testing.

glob_BST_path_list_raw=glob.glob("/databf/nenufar/LT11/*/*/*CAS_A_TRACKING/*BST.fits")
print(len(glob_BST_path_list_raw),glob_BST_path_list_raw)
# Here are pointing data files.


# for fits in glob_BST_path_list_raw:
#     for md in miss_days not in fits:
#         glob_BST_path_list = fits

glob_spectra_path_list_0 = [fits for fits in glob_spectra_path_list_raw if "202307" not in fits]
glob_spectra_path_list = [fits for fits in glob_spectra_path_list_0 if "202302" not in fits]
glob_spectra_path_list_1_a = [fits for fits in glob_spectra_path_list if "202405" not in fits]
glob_spectra_path_list_1_b = [fits for fits in glob_spectra_path_list_1_a if "202407" not in fits]

glob_BST_path_list_0 = [fits for fits in glob_BST_path_list_raw if "202307" not in fits]
glob_BST_path_list = [fits for fits in glob_BST_path_list_0 if "202302" not in fits]
glob_BST_path_list_1a = [fits for fits in glob_BST_path_list if "202405" not in fits]
glob_BST_path_list_1b = [fits for fits in glob_BST_path_list_1a if "202407" not in fits]

print(len(glob_spectra_path_list_1_b),glob_spectra_path_list_1_b)
print(len(glob_BST_path_list_1b),glob_BST_path_list_1b)

# The output of the cell now is here: All CasA <_0.spectra> data, and correspond pointing <BST.fits> data's paths。
# If using high freuqncy line，0.spectra，please use glob_spectra_path_list， glob_BST_path_list。
# If using high freuqncy line，1.spectra，please use glob_spectra_path_list_1_b， glob_BST_path_list_1b。

351 ['/databf/nenufar-tf/LT11/2022/12/20221202_100000_20221202_101000_CAS_A_TRACKING/CAS_A_TRACKING_20221202_100036_1.spectra', '/databf/nenufar-tf/LT11/2022/12/20221202_135000_20221202_140000_CAS_A_TRACKING/CAS_A_TRACKING_20221202_135036_1.spectra', '/databf/nenufar-tf/LT11/2022/12/20221205_100000_20221205_101000_CAS_A_TRACKING/CAS_A_TRACKING_20221205_100036_1.spectra', '/databf/nenufar-tf/LT11/2022/12/20221205_135000_20221205_140000_CAS_A_TRACKING/CAS_A_TRACKING_20221205_135036_1.spectra', '/databf/nenufar-tf/LT11/2022/12/20221207_100000_20221207_101000_CAS_A_TRACKING/CAS_A_TRACKING_20221207_100036_1.spectra', '/databf/nenufar-tf/LT11/2022/12/20221207_142000_20221207_143000_CAS_A_TRACKING/CAS_A_TRACKING_20221207_142036_1.spectra', '/databf/nenufar-tf/LT11/2022/12/20221208_101000_20221208_102000_CAS_A_TRACKING/CAS_A_TRACKING_20221208_101036_1.spectra', '/databf/nenufar-tf/LT11/2022/12/20221208_142000_20221208_143000_CAS_A_TRACKING/CAS_A_TRACKING_20221208_142036_1.spectra', '/databf/ne

In [ ]:
#This is to make plots WITHOUT CORRECTION

# Parameters
f_min = 35 * u.MHz  # Minimum frequency with unit
f_max = 38 * u.MHz  # Maximum frequency with unit
t_step = 10  # Time step in seconds for averaging
f_step = 3000  # Frequency step in kHz for rebinning
Polar_stokes = "Q/I"  # Stokes parameter to use

# Master dictionaries to store results for dynamic Polar_stokes
stokes_data = {
    "get_data": {},  # Stores the raw Stokes data for each Polar_stokes
    "get_fin": {},  # Stores the final weighted averages
    "errors": {},  # Stores individual errors for each data point
    "point_ele_fin": {},  # Stores elevation values
}

# Dictionaries for other intermediate results
sp_dataload = {}
point_dataload = {}
point = {}
point_ele = {}  # Elevation values
wrong_freq_date = {}  # To store dates with invalid frequency ranges

# Process each spectra path
for i in range(len(glob_spectra_path_list)):
    # Load Spectra data
    sp_dataload[i] = Spectra(glob_spectra_path_list[i])
    sp_dataload[i].pipeline.parameters["remove_channels"] = [0, 1, -1]

    # Load Pointing data
    point_dataload[i] = Pointing.from_bst(BST(glob_BST_path_list[i]), analog=False)
    
    # Check frequency validity and skip if not met
    if f_min < sp_dataload[i].frequency_min.to(u.MHz):  # Ensure compatible units
        wrong_freq_date[i] = sp_dataload[i].time_min
        continue
    if f_max > sp_dataload[i].frequency_max.to(u.MHz):  # Ensure compatible units
        wrong_freq_date[i] = sp_dataload[i].time_min
        continue

    # Retrieve Stokes parameter data
    stokes_data["get_data"].setdefault(Polar_stokes, {})[i] = sp_dataload[i].get(
        fmin=f_min,
        fmax=f_max,
        stokes=Polar_stokes,  # Use dynamic Polar_stokes variable
        rebin_dt=t_step * u.s,
        rebin_df=f_step * u.kHz,
    )
    del sp_dataload[i]  # Free memory for Spectra data

    # Get pointing data at mid-time
    mid_time = stokes_data["get_data"][Polar_stokes][i].time[int(len(stokes_data["get_data"][Polar_stokes][i].time) / 2)]
    point[i] = point_dataload[i][mid_time]
    del point_dataload[i]  # Free memory for Pointing data
    point_ele[i] = point[i].horizontal_coordinates.alt.deg

    # Calculate weights and filter data
    data = stokes_data["get_data"][Polar_stokes][i]._data
    weights = (data >= -0.5) & (np.absolute(data) < 0.5)  # Exclude values < -0.5 and large absolute values

    # Calculate weighted mean
    weighted_mean = np.average(np.absolute(data), weights=weights)
    stokes_data["get_fin"].setdefault(Polar_stokes, {})[i] = weighted_mean

    # Calculate weighted standard deviation (error)
    weighted_variance = np.average((np.absolute(data) - weighted_mean) ** 2, weights=weights)
    stokes_data["errors"].setdefault(Polar_stokes, {})[i] = np.sqrt(weighted_variance)

    # Store elevation value
    stokes_data["point_ele_fin"].setdefault(Polar_stokes, {})[i] = point_ele[i].data[0]

    # Free memory for the current Spectra data
    del stokes_data["get_data"][Polar_stokes][i]

# Convert results to lists dynamically for plotting
values = list(stokes_data["get_fin"][Polar_stokes].values())  # Final weighted averages
elevation_values = list(stokes_data["point_ele_fin"][Polar_stokes].values())  # Final elevation values
errors_list = [stokes_data["errors"][Polar_stokes][i] * 100 for i in range(len(stokes_data["errors"][Polar_stokes]))]  # Convert errors to percentages

# Convert Stokes values to percentages
values_percentage = [value * 100 for value in values]

# Dynamic title for the plot
plot_title = f"Cas_A {f_min.value}-{f_max.value} {f_min.unit} NenuFAR {Polar_stokes} vs Elevation"

# Create scatter plot with individual error bars
plt.errorbar(elevation_values, values_percentage, yerr=errors_list, fmt='o', capsize=5, label=f"Cas_A {Polar_stokes}")
plt.scatter(elevation_values, values_percentage, color='red', label='Data Points')

# Add labels, title, and legend
plt.xlabel('Elevation (Degrees)')
plt.ylabel(f"{Polar_stokes} (Percentage)")
plt.title(plot_title)
plt.legend()

# Show the plot
plt.show()

2024-12-07 22:26:08 | INFO: Reading /databf/nenufar-tf/LT11/2022/12/20221202_100000_20221202_101000_CAS_A_TRACKING/CAS_A_TRACKING_20221202_100036_0.spectra...
2024-12-07 22:26:08 | INFO: 	/databf/nenufar-tf/LT11/2022/12/20221202_100000_20221202_101000_CAS_A_TRACKING/CAS_A_TRACKING_20221202_100036_0.spectra has been correctly parsed.
2024-12-07 22:26:08 | INFO: Checking for missing data (can take up to 1 min)...
2024-12-07 22:26:16 | INFO: 	There are 0/564 blocks containing missing data and/or wrong time information.
2024-12-07 22:26:16 | INFO: Computing time-frequency axes...
2024-12-07 22:26:16 | INFO: Re-organize data into Jones matrices...
2024-12-07 22:26:42 | INFO: Data loaded.
2024-12-07 22:26:42 | INFO: Selecting times (between 2022-12-02T10:01:12.000 and 2022-12-02T10:09:34.687)...
2024-12-07 22:26:42 | INFO: 	47940 time samples selected.
2024-12-07 22:26:42 | INFO: Selecting frequencies (between 30.0 MHz and 31.0 MHz)...
2024-12-07 22:26:42 | INFO: 	384 frequency samples selec

KeyboardInterrupt: 

In [ ]:
#This is to make plots WITH POLAR CORRECTION

# Parameters
f_min = 74 * u.MHz  # Minimum frequency with unit
f_max = 77 * u.MHz  # Maximum frequency with unit
t_step = 10  # Time step in seconds for averaging
f_step = 3000  # Frequency step in kHz for rebinning
Polar_stokes = "Q/I"  # Stokes parameter to use

# Parameters for SkyCoord
skycoord = SkyCoord(ra=23.3907 * u.hourangle, dec=58.8153 * u.deg, frame='icrs')  # Example: Cas A

# Master dictionaries to store results for dynamic Polar_stokes
stokes_data = {
    "get_data": {},  # Stores the raw Stokes data for each Polar_stokes
    "get_fin": {},  # Stores the final weighted averages
    "errors": {},  # Stores individual errors for each data point
    "point_ele_fin": {},  # Stores elevation values
}

# Dictionaries for other intermediate results
sp_dataload = {}
point_dataload = {}
point = {}
point_ele = {}  # Elevation values
wrong_freq_date = {}  # To store dates with invalid frequency ranges

# Process each spectra path
for i in range(len(glob_spectra_path_list_1_b)):

    # Load Spectra data
    sp_dataload[i] = Spectra(glob_spectra_path_list[i])
    sp_dataload[i].pipeline.parameters["remove_channels"] = [0, 1, -1]
    sp_dataload[i].pipeline.insert(TFTask.correct_polarization(),1) # insert TFTask.correct_polarization() task in the pipeline
    # sp_dataload[0].pipeline.info()

    # Load Pointing data
    point_dataload[i] = Pointing.from_bst(BST(glob_BST_path_list[i]), analog=False)

    # Check frequency validity and skip if not met
    if f_min < sp_dataload[i].frequency_min.to(u.MHz):  # Ensure compatible units
        wrong_freq_date[i] = sp_dataload[i].time_min
        continue
    if f_max > sp_dataload[i].frequency_max.to(u.MHz):  # Ensure compatible units
        wrong_freq_date[i] = sp_dataload[i].time_min
        continue

    # Retrieve Stokes parameter data
    stokes_data["get_data"].setdefault(Polar_stokes, {})[i] = sp_dataload[i].get(
        fmin=f_min,
        fmax=f_max,
        stokes=Polar_stokes,  # Use dynamic Polar_stokes variable
        rebin_dt=t_step * u.s,
        rebin_df=f_step * u.kHz,
        dreambeam_dt=t_step,  # Use rebin_dt as the time resolution
        dreambeam_skycoord=skycoord,  # Pass the SkyCoord object
        dreambeam_parallactic=True  # Enable parallactic angle correction
    )
    del sp_dataload[i]  # Free memory for Spectra data

    # Get pointing data at mid-time
    mid_time = stokes_data["get_data"][Polar_stokes][i].time[int(len(stokes_data["get_data"][Polar_stokes][i].time) / 2)]
    point[i] = point_dataload[i][mid_time]
    del point_dataload[i]  # Free memory for Pointing data
    point_ele[i] = point[i].horizontal_coordinates.alt.deg

    # Calculate weights and filter data
    data = stokes_data["get_data"][Polar_stokes][i]._data
    weights = (data >= -0.2) & (np.absolute(data) < 0.2)  # Exclude values < -0.5 and large absolute values
    
    # Check for non-zero weights
    if weights.sum() == 0:
        print(f"Skipping index {i}: No valid data points for weighting")
        stokes_data["get_fin"].setdefault(Polar_stokes, {})[i] = None  # Handle missing data
        stokes_data["errors"].setdefault(Polar_stokes, {})[i] = None
        continue

    # Calculate weighted mean
    weighted_mean = np.average(np.absolute(data), weights=weights)
    stokes_data["get_fin"].setdefault(Polar_stokes, {})[i] = weighted_mean

    # Calculate weighted standard deviation (error)
    weighted_variance = np.average((np.absolute(data) - weighted_mean) ** 2, weights=weights)
    stokes_data["errors"].setdefault(Polar_stokes, {})[i] = np.sqrt(weighted_variance)

    # Store elevation value
    stokes_data["point_ele_fin"].setdefault(Polar_stokes, {})[i] = point_ele[i].data[0]

    # Free memory for the current Spectra data
    del stokes_data["get_data"][Polar_stokes][i]

# Filter out None values from values
values = [value for value in stokes_data["get_fin"][Polar_stokes].values() if value is not None]

# Convert valid values to percentages
values_percentage = [value * 100 for value in values]

# Ensure elevation_values aligns with filtered values
elevation_values = [
    stokes_data["point_ele_fin"][Polar_stokes][key]
    for key in stokes_data["get_fin"][Polar_stokes]
    if stokes_data["get_fin"][Polar_stokes][key] is not None
]

# Ensure errors_list aligns with filtered values
errors_list = [
    stokes_data["errors"][Polar_stokes][key] * 100
    for key in stokes_data["get_fin"][Polar_stokes]
    if stokes_data["get_fin"][Polar_stokes][key] is not None
]

# Dynamic title for the plot
plot_title = f"Cas_A {f_min.value}-{f_max.value} {f_min.unit} NenuFAR {Polar_stokes} vs Elevation"

# Create scatter plot with individual error bars
plt.errorbar(elevation_values, values_percentage, yerr=errors_list, fmt='x', capsize=5, label=f"Cas_A {Polar_stokes}")
plt.scatter(elevation_values, values_percentage, color='red', label='Data Points')

# Add labels, title, and legend
plt.xlabel('Elevation (Degrees)')
plt.ylabel(f"{Polar_stokes} (Percentage)")
plt.title(plot_title)
plt.legend()

# Show the plot
plt.show()